### データセットのDL

In [ ]:
# hanaya-san/seed の feat/r4-workブランチをクローンしてくる
!git clone -b feat/r4-work https://github.com/hanaya-san/seed.git  

# 今回使用するデータを直下にコピーしてくる
!cp -r './seed/sandbox/r4-util/lecture-materials/data/titanic/' './data'

## タイタニックデータセット
> タイタニック号沈没事故（タイタニックごうちんぼつじこ）とは、1912年4月14日の夜から4月15日の朝にかけて、イギリス・サウサンプトンからアメリカ合衆国・ニューヨーク行きの処女航海中の4日目に、北大西洋で起きた。当時世界最大の客船であったタイタニックは、1912年4月14日の23時40分（事故現場時間）に氷山に衝突した時には2,224人を乗せていた。事故が起きてから2時間40分後の翌4月15日の2時20分に沈没し、1,513人が亡くなった。これは1912年当時、海難事故の最大死者数であった。

## モチベーション
#### 何をするの？
- どのような人が生き残る可能性が高かったのか分析する。
- タイタニック号乗車客の生存/死亡を予測する機械学習モデルを作る。

#### 何が身につくの？
- Pythonの基礎(配列、DataFrameの扱い)
- ２値分類

In [ ]:
!ls './data'

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

### Numpy
***
Pythonにおいて数値計算を効率的に行うための拡張モジュールである。  
効率的な数値計算を行うための型付きの多次元配列（例えばベクトルや行列などを表現できる）のサポートをPythonに加えるとともに、それらを操作するための大規模な高水準の数学関数ライブラリを提供する。
<br>
<br>
### Pandas
***
Pythonにおいて、データ解析を支援する機能を提供するライブラリである。  
特に、数表および時系列データを操作するためのデータ構造と演算を提供する。  
メモリ内のデータ構造とその他のフォーマットのデータ間で相互に読み書きするためのツール群。  
フォーマット例: CSV、テキストファイル、Excel、SQLデータベース、HDF5フォーマットなど。


####  データセットの読み込み
- train.csv (トレーニングデータ)
- test.csv (テストデータ)
- gender_submission.csv (テストデータの正解ラベル)
***

##### トレーニングデータ
- 機械学習モデルを構築するために使用する。
- 乗客それぞれの生存に関する結果を正解データとして持っている。

##### テストデータ
- 作成したモデルの予測評価を検証するデータ。
- テストデータは、各乗客について生存(正解)を持っていない。  
　→ これを予測するのが今回の目的

##### テストデータの正解ラベル
- テストデータを予測評価した際に、答え合わせする為のデータ

In [ ]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
test_answer_df = pd.read_csv('./data/gender_submission.csv')

#### データの中身を確認
- train.csv
  - 891名の乗客データ
- test.csv
  - 418名の乗客データ
- gender_submission.csv
  - テストデータ(418名)の生存/死亡データ

In [ ]:
print('== Training Data ==')
display(train_df)
print('== Test Data ==')
display(test_df)
print('== Test Answer Data ==')
display(test_answer_df)

In [ ]:
'''
上記だとテーブルが大量に表示されるので、
各テーブル上位3件のみ表示して見やすくする。
'''
print('== Training Data ==')
display(train_df.head(3))
print('== Test Data ==')
display(test_df.head(3))
print('== Test Answer Data ==')
display(test_answer_df.head(3))

#### データセットの項目説明
***

|変数名|説明|備考|
|:--|:--|:--|
|PassengerId|固有ID||
|Survived|生存フラグ|0:死亡, 1:生存|
|Pclass|チケットの階級|1:高い, 2:中, 3:低い|
|Name|名前||
|Sex|性別||
|Age|年齢||
|SibSp|乗船している兄弟・配偶者の数||
|Parch|乗船している両親・子供の数||
|Ticket|チケット番号||
|Fare|運賃||
|Cabin|客室番号||
|Embarked|乗船した港名|C:Cherbourg, Q:Queenstown, S:Southampton|

#### データの欠損値を確認

In [ ]:
print('check NaN (train)')
display(train_df.isnull().sum())

print('check NaN (Test)')
display(test_df.isnull().sum())

#### 基礎統計量を表示する
***
(全て、文字列、欠損値は計算対象外とする)
- count : 個数
- mean : 平均値
- std : 標準偏差 (データのバラつき具合)
- min : 最小値
- 25% : 25パーセンタイル（第一四分位数）
- 50% : 中央値
- 75% : 75パーセンタイル（第三四分位数）
- max : 最大値

In [ ]:
train_df.describe()

#### 統計量の可視化

In [ ]:
import matplotlib.pyplot as plt

### matplotlib
***
Matplotlibは、Pythonおよびその科学計算用ライブラリNumPyのためのグラフ描画ライブラリである。  
描画したグラフを各種形式の画像（各種ベクトル画像形式も含む）として保存することもできる。

In [ ]:
# データの整形

survive_df = train_df[train_df['Survived'] == 1]
dead_df = train_df[train_df['Survived'] == 0]

In [ ]:
'''
データの整形
  df['Age']: DataFrameのAge列だけを抽出
  value_counts: 配列に存在する同じ値(年齢)のデータをカウント
  sort_index: Index(今回は年齢)の昇順ソート
'''
view_survive_data = survive_df['Age'].value_counts().sort_index()
view_dead_data = dead_df['Age'].value_counts().sort_index()

plt.figure(figsize=(14, 6)) # グラフ描画領域のサイズを指定
plt.title('Survived by age') # グラフタイトルを指定
plt.bar(np.arange(len(view_survive_data)), view_survive_data, alpha=0.6, color='blue') # 棒グラフを描画
plt.bar(np.arange(len(view_dead_data)), view_dead_data, alpha=0.6, color='red') # 棒グラフを描画
plt.legend(['Survived', 'Dead']) # グラフの凡例を指定
plt.ylabel('count') # y軸のラベル名を指定
plt.xlabel('age') # x軸ラベルを指定
plt.show() # グラフを描画

In [ ]:
'''
データの整形
  df['Sex']: DataFrameのSex列だけを抽出
  value_counts: 配列に存在する同じ値(性別)のデータをカウント
  sort_index: Index(今回は性別)の昇順ソート
'''
target_variable = 'Sex'

view_survive_data = survive_df[target_variable].value_counts().sort_index()
view_dead_data = dead_df[target_variable].value_counts().sort_index()

plt.figure(figsize=(14, 6)) # グラフ描画領域のサイズを指定
plt.title('Survived by {}'.format(target_variable)) # グラフタイトルを指定
plt.bar(np.arange(len(view_survive_data)), view_survive_data, alpha=0.6, color='blue') # 棒グラフを描画
plt.bar(np.arange(len(view_dead_data)), view_dead_data, alpha=0.6, color='red') # 棒グラフを描画
plt.legend(['Survived', 'Dead']) # グラフの凡例を指定
plt.ylabel('count') # y軸のラベル名を指定
plt.xlabel(target_variable) # x軸ラベルを指定
plt.xticks(np.arange(len(view_survive_data)), view_survive_data.index)
plt.show() # グラフを描画

In [ ]:
'''
データの整形
  df['Pclass']: DataFrameのSex列だけを抽出
  value_counts: 配列に存在する同じ値(チケット番号)のデータをカウント
  sort_index: Index(今回はチケット番号)の昇順ソート
'''
target_variable = 'Pclass'

view_survive_data = survive_df[target_variable].value_counts().sort_index()
view_dead_data = dead_df[target_variable].value_counts().sort_index()

plt.figure(figsize=(14, 6)) # グラフ描画領域のサイズを指定
plt.title('Survived by {}'.format(target_variable)) # グラフタイトルを指定
plt.bar(np.arange(len(view_survive_data)), view_survive_data, alpha=0.6, color='blue') # 棒グラフを描画
plt.bar(np.arange(len(view_dead_data)), view_dead_data, alpha=0.6, color='red') # 棒グラフを描画
plt.legend(['Survived', 'Dead']) # グラフの凡例を指定
plt.ylabel('count') # y軸のラベル名を指定
plt.xlabel(target_variable) # x軸ラベルを指定
plt.xticks(np.arange(len(view_survive_data)), view_survive_data.index)
plt.show() # グラフを描画

In [ ]:
train_df['Sex'] = np.where(train_df['Sex']=='male', 0, 1)
test_df['Sex'] = np.where(test_df['Sex']=='male', 0, 1)

train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df

In [ ]:
X_train = train_df.drop(["Name", "PassengerId", "Survived", "Cabin", "Ticket", "Embarked"], axis=1).copy() # 不要な列を削除
Y_train = train_df['Survived'] # Y_trainは、df_trainのSurvived列
X_test  = test_df.drop(["Name", "PassengerId", "Cabin", "Ticket", "Embarked"], axis=1).copy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 学習と予測を行う
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)
submission = pd.DataFrame({
        'PassengerId': test_df['PassengerId'],
        'Survived': test_answer_df['Survived'],
        'Pred': Y_pred
    })
# submission.to_csv('submission.csv', index=False)
display(submission)
acc = round(len(submission[submission['Survived']==submission['Pred']]) / len(submission), 4)
print('accuracy: {}'.format(acc))

In [ ]:
X_train